# 속도 제한을 처리하는 방법

OpenAI API를 반복적으로 호출하면 `429: 'Too Many Requests'` 또는 `RateLimitError`라는 오류 메시지가 나타날 수 있습니다. 이러한 오류 메시지는 API의 속도 제한을 초과하여 발생합니다.

이 가이드는 속도 제한 오류를 방지하고 처리하기 위한 팁을 공유합니다.

속도 제한 오류를 방지하기 위해 병렬 요청을 조절하는 예제 스크립트를 보려면 [api_request_parallel_processor.py](api_request_parallel_processor.py)를 참조하십시오.

## 속도 제한이 존재하는 이유

속도 제한은 API에 대한 일반적인 관행이며 몇 가지 다른 이유로 적용됩니다.

- 첫째, API의 남용 또는 오용으로부터 보호합니다. 예를 들어 악의적인 행위자는 API에 과부하를 일으키거나 서비스를 중단시키려는 시도로 API에 요청을 플러딩할 수 있습니다. 속도 제한을 설정함으로써 OpenAI는 이러한 종류의 활동을 방지할 수 있습니다.
- 둘째, 속도 제한은 모든 사람이 API에 공정하게 액세스할 수 있도록 합니다. 한 개인이나 조직이 과도한 수의 요청을 하면 다른 모든 사람의 API가 중단될 수 있습니다. 단일 사용자가 만들 수 있는 요청 수를 제한함으로써 OpenAI는 모든 사람이 속도 저하 없이 API를 사용할 수 있는 기회를 갖도록 보장합니다.
- 마지막으로 속도 제한은 OpenAI가 인프라의 총 부하를 관리하는 데 도움이 될 수 있습니다. API에 대한 요청이 급격히 증가하면 서버에 부담을 주고 성능 문제를 일으킬 수 있습니다. 속도 제한을 설정함으로써 OpenAI는 모든 사용자에게 원활하고 일관된 경험을 유지하는 데 도움을 줄 수 있습니다.

속도 제한에 도달하는 것은 실망스러울 수 있지만 속도 제한은 사용자를 위해 API의 안정적인 작동을 보호하기 위해 존재합니다.

## 기본 속도 제한

2023년 1월부터 기본 속도 제한은 다음과 같습니다.:

<table>
<thead>
  <tr>
    <th></th>
    <th>Text Completion &amp; Embedding endpoints</th>
    <th>Code &amp; Edit endpoints</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>Free trial users</td>
    <td>
        <ul>
            <li>20 requests / minute</li>
            <li>150,000 tokens / minute</li>
        </ul>
    </td>
    <td>
        <ul>
            <li>20 requests / minute</li>
            <li>150,000 tokens / minute</li>
        </ul>
    </td>
  </tr>
  <tr>
    <td>Pay-as-you-go users (in your first 48 hours)</td>
    <td>
        <ul>
            <li>60 requests / minute</li>
            <li>250,000 davinci tokens / minute (and proportionally more for cheaper models)</li>
        </ul>
    </td>
    <td>
        <ul>
            <li>20 requests / minute</li>
            <li>150,000 tokens / minute</li>
        </ul>
    </td>
  </tr>
  <tr>
    <td>Pay-as-you-go users (after your first 48 hours)</td>
    <td>
        <ul>
            <li>3,000 requests / minute</li>
            <li>250,000 davinci tokens / minute (and proportionally more for cheaper models)</li>
        </ul>
    </td>
    <td>
        <ul>
            <li>20 requests / minute</li>
            <li>150,000 tokens / minute</li>
        </ul>
    </td>
  </tr>
</tbody>
</table>

For reference, 1,000 tokens is roughly a page of text.

### 기타 속도 제한 리소스

다른 리소스에서 OpenAI의 속도 제한에 대해 자세히 읽어보세요.:

- [Guide: Rate limits](https://beta.openai.com/docs/guides/rate-limits/overview)
- [Help Center: Is API usage subject to any rate limits?](https://help.openai.com/en/articles/5955598-is-api-usage-subject-to-any-rate-limits)
- [Help Center: How can I solve 429: 'Too Many Requests' errors?](https://help.openai.com/en/articles/5955604-how-can-i-solve-429-too-many-requests-errors)

### Requesting a rate limit increase

조직의 속도 제한을 늘리려면 다음 양식을 작성하십시오.:

- [OpenAI Rate Limit Increase Request form](https://forms.gle/56ZrwXXoxAN1yt6i9)


## 속도 제한 오류의 예

API 요청이 너무 빨리 전송되면 속도 제한 오류가 발생합니다. OpenAI Python 라이브러리를 사용하는 경우 다음과 같이 표시됩니다.:

```
RateLimitError: Rate limit reached for default-codex in organization org-{id} on requests per min. Limit: 20.000000 / min. Current: 24.000000 / min. Contact support@openai.com if you continue to have issues or if you’d like to request an increase.
```

아래는 속도 제한 오류를 트리거하는 예제 코드입니다.

In [ ]:
import openai  # for making OpenAI API requests

# request a bunch of completions in a loop
for _ in range(100):
    openai.Completion.create(
        model="code-cushman-001",
        prompt="def magic_function():\n\t",
        max_tokens=10,
    )


## 속도 제한 오류를 피하는 방법

### 지수 백오프로 재시도

속도 제한 오류를 방지하는 한 가지 쉬운 방법은 무작위 지수 백오프로 요청을 자동으로 재시도하는 것입니다. 지수 백오프로 재시도한다는 것은 속도 제한 오류에 도달했을 때 짧은 절전 모드를 수행한 다음 실패한 요청을 다시 시도하는 것을 의미합니다. 요청이 여전히 실패하면 수면 시간이 증가하고 프로세스가 반복됩니다. 이것은 요청이 성공하거나 최대 재시도 횟수에 도달할 때까지 계속됩니다.

이 접근 방식에는 다음과 같은 많은 이점이 있습니다.

- 자동 재시도는 충돌이나 데이터 손실 없이 속도 제한 오류에서 복구할 수 있음을 의미합니다.
- 지수 백오프는 첫 번째 재시도를 빠르게 시도할 수 있음을 의미하며 처음 몇 번의 재시도가 실패할 경우 더 긴 지연의 이점을 누릴 수 있습니다.
- 지연에 랜덤 지터를 추가하면 동시에 모든 타격에서 재시도할 수 있습니다.

실패한 요청은 분당 한도에 영향을 미치므로 계속해서 요청을 다시 보내면 작동하지 않습니다.

다음은 몇 가지 예시 솔루션입니다.

#### 예제 #1: Tenacity 라이브러리 사용

[Tenacity](https://tenacity.readthedocs.io/en/latest/) 는 Python으로 작성된 Apache 2.0 라이센스 범용 재시도 라이브러리로 거의 모든 항목에 재시도 동작을 추가하는 작업을 단순화합니다.

요청에 지수 백오프를 추가하려면 `tenacity.retry` [decorator](https://peps.python.org/pep-0318/)를 사용할 수 있습니다. 다음 예제에서는 `tenacity.wait_random_exponential` 함수를 사용하여 임의의 지수 백오프를 요청에 추가합니다.

Tenacity 라이브러리는 타사 도구이며 OpenAI는 안정성이나 보안을 보장하지 않습니다.

In [1]:
import openai  # for OpenAI API calls
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff


@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return openai.Completion.create(**kwargs)


completion_with_backoff(model="text-davinci-002", prompt="Once upon a time,")


<OpenAIObject text_completion id=cmpl-5oowO391reUW8RGVfFyzBM1uBs4A5 at 0x10d8cae00> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " a little girl dreamed of becoming a model.\n\nNowadays, that dream"
    }
  ],
  "created": 1662793900,
  "id": "cmpl-5oowO391reUW8RGVfFyzBM1uBs4A5",
  "model": "text-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 5,
    "total_tokens": 21
  }
}

#### 예 #2: 백오프 라이브러리 사용

백오프 및 재시도를 위한 함수 데코레이터를 제공하는 또 다른 라이브러리는 [backoff](https://pypi.org/project/backoff/)입니다.

Tenacity와 마찬가지로 백오프 라이브러리는 타사 도구이며 OpenAI는 안정성이나 보안에 대해 보장하지 않습니다.

In [2]:
import backoff  # for exponential backoff
import openai  # for OpenAI API calls


@backoff.on_exception(backoff.expo, openai.error.RateLimitError)
def completions_with_backoff(**kwargs):
    return openai.Completion.create(**kwargs)


completions_with_backoff(model="text-davinci-002", prompt="Once upon a time,")


<OpenAIObject text_completion id=cmpl-5oowPhIdUvshEsF1rBhhwE9KFfI3M at 0x111043680> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " two children lived in a poor country village. In the winter, the temperature would"
    }
  ],
  "created": 1662793901,
  "id": "cmpl-5oowPhIdUvshEsF1rBhhwE9KFfI3M",
  "model": "text-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 5,
    "total_tokens": 21
  }
}

#### 예 3: 수동 백오프 구현

타사 라이브러리를 사용하지 않으려면 자체 백오프 논리를 구현할 수 있습니다.

In [3]:
# imports
import random
import time

import openai

# define a retry decorator
def retry_with_exponential_backoff(
    func,
    initial_delay: float = 1,
    exponential_base: float = 2,
    jitter: bool = True,
    max_retries: int = 10,
    errors: tuple = (openai.error.RateLimitError,),
):
    """Retry a function with exponential backoff."""

    def wrapper(*args, **kwargs):
        # Initialize variables
        num_retries = 0
        delay = initial_delay

        # Loop until a successful response or max_retries is hit or an exception is raised
        while True:
            try:
                return func(*args, **kwargs)

            # Retry on specified errors
            except errors as e:
                # Increment retries
                num_retries += 1

                # Check if max retries has been reached
                if num_retries > max_retries:
                    raise Exception(
                        f"Maximum number of retries ({max_retries}) exceeded."
                    )

                # Increment the delay
                delay *= exponential_base * (1 + jitter * random.random())

                # Sleep for the delay
                time.sleep(delay)

            # Raise exceptions for any errors not specified
            except Exception as e:
                raise e

    return wrapper


@retry_with_exponential_backoff
def completions_with_backoff(**kwargs):
    return openai.Completion.create(**kwargs)


completions_with_backoff(model="text-davinci-002", prompt="Once upon a time,")


<OpenAIObject text_completion id=cmpl-5oowRsCXv3AkUgVJyyo3TQrVq7hIT at 0x111024220> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " a man decided to greatly improve his karma by turning his life around.\n\n"
    }
  ],
  "created": 1662793903,
  "id": "cmpl-5oowRsCXv3AkUgVJyyo3TQrVq7hIT",
  "model": "text-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 5,
    "total_tokens": 21
  }
}

## 속도 제한이 주어진 일괄 처리의 처리량을 최대화하는 방법

사용자의 실시간 요청을 처리하는 경우 백오프 및 재시도는 속도 제한 오류를 피하면서 대기 시간을 최소화하는 훌륭한 전략입니다.

그러나 대기 시간보다 처리량이 더 중요한 대량의 배치 데이터를 처리하는 경우 백오프 및 재시도 외에 수행할 수 있는 몇 가지 다른 작업이 있습니다.

### 요청 사이에 사전에 지연 추가

지속적으로 속도 제한에 도달한 다음 백오프하고 다시 속도 제한에 도달한 다음 다시 백오프하는 경우 재시도해야 하는 요청에서 요청 예산의 상당 부분이 '낭비'될 수 있습니다. 이는 고정 속도 제한이 있는 처리량을 제한합니다.

여기서 한 가지 가능한 해결책은 속도 제한을 계산하고 그 역수에 해당하는 지연을 추가하는 것입니다(예: 속도 제한이 분당 20개 요청인 경우 각 요청에 3~6초의 지연 추가). 이렇게 하면 속도 제한 상한선에 도달하지 않고 낭비되는 요청을 발생시키지 않고 근처에서 운영하는 데 도움이 될 수 있습니다.

#### 요청에 지연을 추가하는 예

In [4]:
# imports
import time
import openai

# Define a function that adds a delay to a Completion API call
def delayed_completion(delay_in_seconds: float = 1, **kwargs):
    """Delay a completion by a specified amount of time."""

    # Sleep for the delay
    time.sleep(delay_in_seconds)

    # Call the Completion API and return the result
    return openai.Completion.create(**kwargs)


# Calculate the delay based on your rate limit
rate_limit_per_minute = 20
delay = 60.0 / rate_limit_per_minute

delayed_completion(
    delay_in_seconds=delay,
    model="text-davinci-002",
    prompt="Once upon a time,"
)


<OpenAIObject text_completion id=cmpl-5oowVVZnAzdCPtUJ0rifeamtLcZRp at 0x11b2c7680> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " there was an idyllic little farm that sat by a babbling brook"
    }
  ],
  "created": 1662793907,
  "id": "cmpl-5oowVVZnAzdCPtUJ0rifeamtLcZRp",
  "model": "text-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 5,
    "total_tokens": 21
  }
}



### 일괄 요청

OpenAI API에는 분당 요청 및 분당 토큰에 대한 별도의 제한이 있습니다.

분당 요청 제한에 도달했지만 분당 토큰에 여유가 있는 경우 여러 작업을 각 요청에 일괄 처리하여 처리량을 늘릴 수 있습니다. 이렇게 하면 특히 더 작은 모델에서 분당 더 많은 토큰을 처리할 수 있습니다.

프롬프트 일괄 전송은 문자열 목록을 단일 문자열 대신 `prompt` 매개변수에 전달한다는 점을 제외하면 일반 API 호출과 정확히 동일하게 작동합니다.

**경고:** 응답 개체는 프롬프트 순서대로 완료를 반환하지 않을 수 있으므로 항상 '인덱스' 필드를 사용하여 응답을 프롬프트에 다시 일치시켜야 합니다.

#### 일괄 처리가 없는 예

In [5]:
import openai  # for making OpenAI API requests


num_stories = 10
prompt = "Once upon a time,"

# serial example, with one story completion per request
for _ in range(num_stories):
    response = openai.Completion.create(
        model="curie",
        prompt=prompt,
        max_tokens=20,
    )

    # print story
    print(prompt + response.choices[0].text)


Once upon a time, before there were grandiloquent tales of the massacre at Fort Mims, there were stories of
Once upon a time, a full-sized search and rescue was created. However, CIDIs are the addition of requiring
Once upon a time, Schubert was hot with the films. “Schubert sings of honey, flowers,
Once upon a time, you could watch these films on your VCR, sometimes years after their initial theatrical release, and there
Once upon a time, there was a forest. In that forest, the forest animals ruled. The forest animals had their homes
Once upon a time, there were two programs that complained about false positive scans. Peacock and Midnight Manager alike, only
Once upon a time, a long, long time ago, tragedy struck. it was the darkest of nights, and there was
Once upon a time, when Adam was a perfect little gentleman, he was presented at Court as a guarantee of good character.
Once upon a time, Adam and Eve made a mistake. They ate the fruit from the tree of immortality and split the co

#### 일괄 처리의 예

In [6]:
import openai  # for making OpenAI API requests


num_stories = 10
prompts = ["Once upon a time,"] * num_stories

# batched example, with 10 stories completions per request
response = openai.Completion.create(
    model="curie",
    prompt=prompts,
    max_tokens=20,
)

# match completions to prompts by index
stories = [""] * len(prompts)
for choice in response.choices:
    stories[choice.index] = prompts[choice.index] + choice.text

# print stories
for story in stories:
    print(story)


Once upon a time, there were two sisters, Eliza Pickering and Ariana 'Ari' Lucas. When these lovely
Once upon a time, Keene was stung by a worm — actually, probably a python — snaking through his leg
Once upon a time, there was a professor of physics during the depression. It was difficult, during this time, to get
Once upon a time, before you got sick, you told stories to all and sundry, and your listeners believed in you
Once upon a time, there was one very old nice donkey. He was incredibly smart, in a very old, kind of
Once upon a time, the property of a common lodging house was a common cup for all the inhabitants. Betimes a constant
Once upon a time, in an unspecified country, there was a witch who had an illegal product. It was highly effective,
Once upon a time, a long time ago, I turned 13, my beautiful dog Duncan swept me up into his jaws like
Once upon a time, as a thoroughly reformed creature from an army of Nazis, he took On Judgement Day myself and his
Once upon a time, C

## 병렬 처리 스크립트 예시

대량의 API 요청을 병렬 처리하기 위한 예제 스크립트를 작성했습니다. [api_request_parallel_processor.py](api_request_parallel_processor.py).

이 스크립트는 몇 가지 편리한 기능을 결합합니다.
- 대용량 작업의 메모리 부족을 방지하기 위해 파일에서 요청 스트리밍
- 처리량을 최대화하기 위해 동시에 요청합니다.
- 속도 제한을 유지하기 위해 요청 및 토큰 사용을 모두 제한합니다.
- 데이터 누락을 방지하기 위해 실패한 요청 재시도
- 오류를 기록하여 요청 문제를 진단합니다.

그대로 사용하거나 필요에 맞게 수정하십시오.